In [21]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [22]:
def compute_removal(topic_a, cur_argn, agg_f, inf_f):
    bag = grad.BAG("../../bags/movie_recommender.bag")
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    arg_name_all = []

    for arg in bag.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")


    # store the initial weights
    arg_initial_weight = []
    # arg_initial_weight_dict = {}
    for arg in bag.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))

    print("All the base score and strength in QBAF:")
    for arg in bag.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))


    difference = []

    # print 1st: before remove cur arg
    print("Initial and final strength (1st):")
    for arg in bag.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))
    topic_arg = bag.arguments[topic_a]
    strength_before_drop_cur = topic_arg.strength
    print(f"strength_before_drop_cur:{strength_before_drop_cur}")

    # remove cur arg
    bag.remove_argument(cur_arg)
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    # print 2nd: after remove cur arg
    print("Initial and final strength (2nd):")
    for arg in bag.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))
    topic_arg = bag.arguments[topic_a]
    strength_after_drop_cur = topic_arg.strength
    print(f"strength_after_drop_cur:{strength_after_drop_cur}")

    # compute the diff of topic arg before and after removing
    difference.append(strength_before_drop_cur-strength_after_drop_cur)
    print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")



    print("----------------------------")


    print(difference)
    attribution = difference[0]
    print(attribution)

    # initializing: reset to original initial weight
    bag = grad.BAG("../../bags/movie_recommender.bag")
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    return attribution

In [23]:
# Computation via Forward Propagation
bag = grad.BAG("../../bags/movie_recommender.bag")

# # QE
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

# DF-QuAD
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# # SD-DF-QUAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.SQDFQUADInfluence(conservativeness=1)

# # EB
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

# # EBT
# agg_f = grad.semantics.modular.TopAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

#returns dictionary of strength values if needed
strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

for arg in bag.arguments.values():
    print((arg.name,arg.strength))

('m', 0.84774307)
('fA', 0.25786000000000003)
('fA1', 0.05)
('fA2', 0.07)
('fD', 0.05)
('fW', 0.02)


In [24]:
# compute attribution for one arg
topic_a = "m"
all_args = bag.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg]=compute_removal(topic_a, cur_arg, agg_f, inf_f)
print(f"attribution: \n{attribution}")

['m', 'fA', 'fA1', 'fA2', 'fD', 'fW']
['fA', 'fA1', 'fA2', 'fD', 'fW']
['m', 'fA', 'fA1', 'fA2', 'fD', 'fW']
['fA', 'fA1', 'fA2', 'fD', 'fW']
['fA1', 'fA2', 'fD', 'fW']
----------------------------
All the base score and strength in QBAF:
('m', 0.79, 0.84774307)
('fA', 0.16, 0.25786000000000003)
('fA1', 0.05, 0.05)
('fA2', 0.07, 0.07)
('fD', 0.05, 0.05)
('fW', 0.02, 0.02)
Initial and final strength (1st):
('m', 0.79, 0.84774307)
('fA', 0.16, 0.25786000000000003)
('fA1', 0.05, 0.05)
('fA2', 0.07, 0.07)
('fD', 0.05, 0.05)
('fW', 0.02, 0.02)
strength_before_drop_cur:0.84774307
Initial and final strength (2nd):
('m', 0.79, 0.7963)
('fA1', 0.05, 0.05)
('fA2', 0.07, 0.07)
('fD', 0.05, 0.05)
('fW', 0.02, 0.02)
strength_after_drop_cur:0.7963
diff=0.051443070000000035
----------------------------
[0.051443070000000035]
0.051443070000000035
['m', 'fA', 'fA1', 'fA2', 'fD', 'fW']
['fA', 'fA1', 'fA2', 'fD', 'fW']
['fA', 'fA2', 'fD', 'fW']
----------------------------
All the base score and strength